In [1]:
import os
import numpy as np
from siphon.ncss import NCSS
from netCDF4 import num2date, Dataset
from datetime import datetime

In [6]:
year = '2017'
month = '04'
day = '05'
hour = '12'
ncss = NCSS('http://thredds.ucar.edu/thredds/ncss/grib/NCEP/GFS/Global_0p5deg/GFS_Global_0p5deg_'
            +year+month+day+'_'+hour+'00.grib2')
query = ncss.query()
query.variables('Geopotential_height_isobaric',
                'Temperature_isobaric',
                'u-component_of_wind_isobaric',
                'v-component_of_wind_isobaric')
query.add_lonlat().vertical_level(50000)
query.time(datetime(int(year),int(month),int(day),int(hour)))
query.lonlat_box(north=65, south=15, east=310, west=220)
data = ncss.get_data(query)

In [2]:
ncss = NCSS('http://nomads.ncdc.noaa.gov/thredds/ncss/grid/namanl/201604/20160416/namanl_218_20160416_1800_000.grb')
now = datetime.utcnow()

# Query for Latest GFS Run
hgt = ncss.query().all_times().accept('netcdf').variables('Geopotential_height','u_wind','v_wind').add_lonlat()

# Actually getting the data
data = ncss.get_data(hgt)

In [3]:
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    Originating_center: US National Weather Service - NCEP(WMC) (7)
    Generating_Model: MESO NAM Model
    Product_Type: Initialized analysis product
    title: US National Weather Service - NCEP(WMC) MESO NAM Model Initialized analysis product
    institution: Center US National Weather Service - NCEP(WMC) (7)
    source: Initialized analysis product
    history: Direct read of GRIB-1 into NetCDF-Java 4 API
    CF:feature_type: GRID
    file_format: GRIB-1
    location: /nomads3_data/raid2/noaaport/merged/namanl/201604/20160416/namanl_218_20160416_1800_000.grb
    _CoordinateModelRunDate: 2016-04-16T18:00:00Z
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (NetcdfCFWriter)
Original Dataset = /nomads3_data/raid2/noaaport/merged/namanl/201604/20160416/namanl_218_20160416_1800_000.grb; Translation Date = Thu Apr 06 11:38:34 EDT 2017
    dimensions(si

In [4]:
#output file
outfile = 'nam_xarray.nc'
try:
    os.remove(outfile)
except OSError:
    pass
dsout = Dataset(outfile, 'w')

#Copy dimensions
for dname, the_dim in data.dimensions.items():
    #print dname, len(the_dim)
    dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

for v_name, varin in data.variables.items():
    #print(v_name, varin)
    outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
    #print varin.datatype
    
    # Copy variable attributes
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    
    # Copy actual data
    outVar[:] = varin[:]

dsout.close()

In [64]:
#print(np.max(data.variables['Temperature_sigma'][:])-273.15)

36.0899902344
